In [1]:
import geopandas as gpd
import numpy as np
from owslib.wms import WebMapService
import matplotlib.pyplot as plt
import shapely
from datetime import datetime
import datetime as dt
import os

In [2]:
os.getcwd()

'/Users/glynnismillhouse/Documents/Winter 2020/Deep-Learning/cs230-project/Code'

## Get and Explore Saved Fire Perimeter Files

In [3]:
fire_perims = gpd.read_file("results.shp")

In [4]:
fire_perims.columns   #see column names, sometimes they change

Index(['ACTIVE', 'AGENCY', 'COMMENTS', 'COMPFIRECD', 'COMPLEXNM', 'COMPPARID',
       'ComplexN_1', 'DATECRNT', 'FIRECODE', 'FIRENAME', 'FIRENUM', 'FIREYEAR',
       'GISACRES', 'INCIDENTID', 'INCIWEBID', 'INCOMPLEX', 'IRWINID', 'LATEST',
       'MAPMETHOD', 'MERGEID', 'OBJECTID', 'PERDATTIME', 'SHAPE_AREA',
       'SHAPE_Ar_1', 'STATE', 'Shape_Ar_2', 'UNITIDOWN', 'UNITIDPROT',
       'active_1', 'agency_1', 'comments_1', 'compParI_1', 'compfire_1',
       'dateCrnt_1', 'fireName_1', 'fireNum_1', 'fireYear_1', 'firecode_1',
       'gisAcres_1', 'inComple_1', 'incident_1', 'inciwebI_1', 'irwinid_1',
       'latest_1', 'mapmetho_1', 'mergeid_1', 'objectid_1', 'perDatTi_1',
       'state_1', 'unitIDOw_1', 'unitIDPr_1', 'geometry'],
      dtype='object')

In [5]:
fire_perims.head()

,ACTIVE,AGENCY,COMMENTS,COMPFIRECD,COMPLEXNM,COMPPARID,ComplexN_1,DATECRNT,FIRECODE,FIRENAME,...,irwinid_1,latest_1,mapmetho_1,mergeid_1,objectid_1,perDatTi_1,state_1,unitIDOw_1,unitIDPr_1,geometry
0,N,BLM,Pulled from EGP REST Service,None,None,None,None,8/22/2019 5:05:50 AM,MNK0,Goshawk,...,{BFEB198B-0812-4365-9C6A-A0DBFFCFAC79},None,None,None,NaN,8/21/2019 4:29:27 PM,UT,None,None,"POLYGON ((-112.91023 40.81809, -112.91019 40.8..."
1,N,NPS,Pulled from EGP REST Service,None,None,None,None,6/30/2019 5:05:30 AM,L9EJ,6 MILE,...,{850dda2b-0d7d-4431-b2e2-32e045f5ecca},None,None,None,NaN,6/28/2019 11:43:31 AM,NV,None,None,"POLYGON ((-114.67735 35.45657, -114.67732 35.4..."
2,N,BLM,Pulled from EGP REST Service,None,None,None,None,8/13/2019 5:05:32 AM,MLY4,Flint Mesa,...,{D1DC65FB-1F52-4F22-8641-6AB763BE1A0F},None,None,None,NaN,8/12/2019 2:16:31 PM,ID,None,None,"POLYGON ((-115.18629 42.89051, -115.18617 42.8..."
3,N,BLM,Perimeter updated using Sentinel 2 image colle...,None,None,None,None,7/17/2019 8:39:52 AM,ME4Z,Tom Cook Slough 2,...,{5C63D91A-0865-4B63-8955-EBECD0C1DFAF},None,None,None,NaN,7/16/2019 3:46:33 PM,AK,None,None,"POLYGON ((-156.49420 65.92108, -156.49421 65.9..."
4,N,BLM,Pulled from EGP REST Service,None,None,None,None,7/11/2019 10:05:58 AM,MD8J,Devil Creek,...,{53e81281-a122-4c15-9395-ef1324528d63},None,None,None,NaN,7/11/2019 9:35:59 AM,ID,None,None,"POLYGON ((-114.93865 42.45531, -114.93869 42.4..."


## Downloading Sentinel Images to Match Perimeter Locations

Sentinel images are pictures of plots of land from the Sentinel Satelite with multiple types of features (vegetation, etc.). The images are taken every 5 days.

In [6]:
wms = WebMapService('http://services.sentinel-hub.com/ogc/wms/45271f49-0dcb-4329-8b35-8b6508fee50b')

### Preliminary exploration of WMS options/features

In [7]:
wms.identification.type
wms.identification.version
wms.identification.title
wms.identification.abstract

'The Copernicus project’s Sentinel satellites are revolutionizing earth observation (EO). Its free, full and open access to data with very short revisit times, high spatial resolution, and good spectral resolution are crucial for many applications. The portfolio of possible products is vast - use-cases of such a service range from plant health monitoring, land and water body change, flood monitoring, disaster mapping and more.However the current gap between Sentinel source data and its end-users is large:• \x90  ESA’s complex Scientific Data Hub• \x90  raster files are compressed with JPEG2000 (13 raster filesfor each product, one per spectral band)• \x90  terabytes of data per week• \x90  additional processing requirementsTackling the data in an old-fashioned way -  offering individual derivative products simply does not work anymore, the associated time and costs are large and defeat most of the major benefits of the Sentinel project.Our approach combines cloud-based GIS technologies

In [8]:
list(wms.contents)  #see list of options for types of features available from Sentinel Satelites

['AGRICULTURE',
 'ARI1',
 'ARI2',
 'ATMOSPHERIC_PENETRATION',
 'B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B09',
 'B10',
 'B11',
 'B12',
 'B8A',
 'BAI',
 'BATHYMETRIC',
 'CHL_RED_EDGE',
 'CRI1',
 'CRI2',
 'EVI',
 'EVI2',
 'FALSE_COLOR',
 'FALSE_COLOR_URBAN',
 'GEOLOGY',
 'GRVI1',
 'LAI_SAVI',
 'MOISTURE_INDEX',
 'MSAVI2',
 'NBR_RAW',
 'NDVI',
 'NDVI_GRAY',
 'NDVI_GREEN_GRAY',
 'NDWI',
 'PSRI',
 'PSRI_NIR',
 'RED_EDGE_NDVI',
 'RE_NDWI',
 'RGB_11_8_3',
 'RGB_4_3_1',
 'RGB_8_11_12',
 'RGB_8_11_4',
 'RGB_8_5_4',
 'RGB_8_6_4',
 'SAVI',
 'SWIR',
 'TRUE_COLOR']

In [ ]:
wms['TRUE_COLOR'].styles 

In [ ]:
[op.name for op in wms.operations]

In [ ]:
wms.getOperationByName('GetMap').methods

In [ ]:
wms.getOperationByName('GetMap').formatOptions

### Image Retrieval

#### Test getting a single image from wms

In [39]:
#Sample format
img = wms.getmap(    layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=(-112.918670, 40.811920, -112.909857, 40.818094),
                     size=(256, 256),
                     format='image/png',
                     time ='2019-08-16/2019-08-21/P1D',                   
                     )

In [40]:
out = open('sentineltest.png', 'wb')  #
out.write(img.read())
out.close()

#### Test getting a single image from wms using one of the perimeter images to find the correct location

In [41]:
#Sample with first element of polygon dataframe
test_polygon = fire_perims['geometry'][0]

ade = test_polygon.simplify(0.1, preserve_topology=True)  #simplify polygon to have fewer sides.
#print(ade)
print(ade.bounds)  #coordinates of bounding box of that polygon.

(-112.91863964999999, 40.812389912000064, -112.91022832499999, 40.81808935800009)


In [42]:
date = fire_perims['perDatTi_1'][0]
l = date.split('/')
date = l[0] + '/' + l[1] + '/' + l[2][:4]
print(date)

8/21/2019


In [43]:
fire_img = wms.getmap(  layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=ade.bounds,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2019-08-21/2019-08-30/P1D',                   
                     )

In [44]:
## Save downloaded image
out = open('sentinel_fire.png', 'wb')  #
out.write(fire_img.read())
out.close()

In [47]:
# get the same image from one month earlier
nofire_img = wms.getmap(  layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=ade.bounds,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2019-07-21/2019-07-30/P1D',                   
                     )

In [48]:
## Save downloaded image
out = open('sentinel_nofire.png', 'wb')  #
out.write(nofire_img.read())
out.close()

#### Loop through and download more images

In [55]:
fire_perims_small = fire_perims[0:15]
fire_perims_small

,ACTIVE,AGENCY,COMMENTS,COMPFIRECD,COMPLEXNM,COMPPARID,ComplexN_1,DATECRNT,FIRECODE,FIRENAME,...,irwinid_1,latest_1,mapmetho_1,mergeid_1,objectid_1,perDatTi_1,state_1,unitIDOw_1,unitIDPr_1,geometry
0,N,BLM,Pulled from EGP REST Service,None,None,None,None,8/22/2019 5:05:50 AM,MNK0,Goshawk,...,{BFEB198B-0812-4365-9C6A-A0DBFFCFAC79},None,None,None,NaN,8/21/2019 4:29:27 PM,UT,None,None,"POLYGON ((-112.91023 40.81809, -112.91019 40.8..."
1,N,NPS,Pulled from EGP REST Service,None,None,None,None,6/30/2019 5:05:30 AM,L9EJ,6 MILE,...,{850dda2b-0d7d-4431-b2e2-32e045f5ecca},None,None,None,NaN,6/28/2019 11:43:31 AM,NV,None,None,"POLYGON ((-114.67735 35.45657, -114.67732 35.4..."
2,N,BLM,Pulled from EGP REST Service,None,None,None,None,8/13/2019 5:05:32 AM,MLY4,Flint Mesa,...,{D1DC65FB-1F52-4F22-8641-6AB763BE1A0F},None,None,None,NaN,8/12/2019 2:16:31 PM,ID,None,None,"POLYGON ((-115.18629 42.89051, -115.18617 42.8..."
3,N,BLM,Perimeter updated using Sentinel 2 image colle...,None,None,None,None,7/17/2019 8:39:52 AM,ME4Z,Tom Cook Slough 2,...,{5C63D91A-0865-4B63-8955-EBECD0C1DFAF},None,None,None,NaN,7/16/2019 3:46:33 PM,AK,None,None,"POLYGON ((-156.49420 65.92108, -156.49421 65.9..."
4,N,BLM,Pulled from EGP REST Service,None,None,None,None,7/11/2019 10:05:58 AM,MD8J,Devil Creek,...,{53e81281-a122-4c15-9395-ef1324528d63},None,None,None,NaN,7/11/2019 9:35:59 AM,ID,None,None,"POLYGON ((-114.93865 42.45531, -114.93869 42.4..."
5,None,None,None,None,None,None,None,None,None,None,...,{73EB1456-74DE-47CC-BE15-57786190CBE2},Y,Mixed Methods,None,NaN,8/14/2019 7:58:00 PM,CA,CAINF,CAINF,"POLYGON ((-118.67056 37.76397, -118.67052 37.7..."
6,None,None,None,None,None,None,None,None,None,None,...,{9748204A-8A87-4B99-9559-6A2BF4A785EE},Y,Hand Sketch,None,NaN,8/15/2019,AZ,AZGCP,AZGCP,"POLYGON ((-112.28026 36.34894, -112.28034 36.3..."
7,None,None,None,None,None,None,None,None,None,None,...,{5A6B5A9F-35CF-4F34-8C66-08F37083B54D},Y,Infrared Image,None,NaN,10/23/2019 7:36:00 PM,CO,CORGF,CORGF,"MULTIPOLYGON (((-105.97126 38.40746, -105.9713..."
8,None,None,None,None,None,None,None,None,None,None,...,{902D8B39-A23A-4D21-AA52-53FBF0493D56},N,Infrared Image,None,NaN,8/21/2019,ID,IDSCF,IDSCF,"POLYGON ((-114.49905 45.31945, -114.49974 45.3..."
9,Y,FS,Pulled from EGP REST Service,None,None,None,None,9/9/2019 10:06:00 AM,MRD4,SOUTH,...,{3C414AA1-6948-4BF3-B6CD-417691D8633E},None,None,None,NaN,9/9/2019 8:00:52 AM,CA,None,None,"POLYGON ((-122.80692 40.11755, -122.80658 40.1..."


In [87]:
# get a single sentinel image for a given time
def getDownloadImage(polygonbounds, timeparams, fireBool, i, date):
    image = wms.getmap(layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polygonbounds,
                     size=(256, 256),  
                     format='image/png',
                     time = timeparams,                   
                     )
    ## Filename includes the date and whether or not there is supposed to be a fire
    filename = 'sentinel' + date + '_' + str(i) + fireBool + '.png'
    out = open(filename, 'wb')  #
    out.write(image.read())
    out.close()

In [88]:
# find a 10 day time interval that matches with when the fire happened for the ith instance of a fire
def getSearchInterval(fire_perims, i):
    # pull date from fire_perims and reformat it
    date = fire_perims['perDatTi_1'][i]
    l = date.split('/')
    date = l[0] + '/' + l[1] + '/' + l[2][:4]
    
    # Reinterpret the date as a datetime and save as a string
    firedate = datetime.strptime(date, '%m/%d/%Y')
    firedate_str = firedate.strftime("%Y-%m-%d")
    
    # Calculate the date that is 10 days in the future to be the end of the time interval we want an image
    interval_enddate = firedate + dt.timedelta(days = 5)  #can reduce if needed
    # interval_enddate = interval_enddate.now()
    interval_enddate_str = interval_enddate.strftime("%Y-%m-%d")
    
    # Construct date interval
    search_interval = firedate_str + '/' + interval_enddate_str + '/' + 'P1D'
    return (search_interval, str(firedate.strftime("%Y")))

In [91]:
def downloadAllBurnImages(fire_perims):
    isFire = 'FIRE'
    for i in range(len(fire_perims)):
        polygon = fire_perims['geometry'][i]
        polygon_simple = polygon.simplify(0.1, preserve_topology=True) 
        polygonbounds = polygon_simple.bounds
        print(polygonbounds)
        (timeparams, fireDate) = getSearchInterval(fire_perims, i)
        getDownloadImage(polygonbounds, timeparams, isFire, i, fireDate)   

In [83]:
os.getcwd()

'/Users/glynnismillhouse/Documents/Winter 2020/Deep-Learning/cs230-project/Code'

In [92]:
downloadAllBurnImages(fire_perims_small)

(-112.91863964999999, 40.812389912000064, -112.91022832499999, 40.81808935800009)
(-114.67782958299995, 35.45431729300003, -114.67610937199998, 35.45656684000006)
(-115.23889540699997, 42.86814297500007, -115.17556736199998, 42.89050995800005)
(-156.50481509099998, 65.91473318100005, -156.48172601999997, 65.92108229700003)
(-114.94553809999996, 42.45284938700007, -114.93865102499996, 42.459096839000054)
(-118.67289747176274, 37.76270054909859, -118.6705585020209, 37.76397451392717)
(-112.29692204899999, 36.33697812490091, -112.23213355099995, 36.365028288901236)
(-106.0376107997079, 38.40721383186162, -105.95160481437547, 38.485281824048705)
(-114.49940764449282, 45.319446634151134, -114.4290369396781, 45.373560754559136)
(-122.82307474699996, 40.08467042800004, -122.77381913999993, 40.12737000000004)
(-122.65411555242974, 40.03150393805106, -122.61360346741226, 40.071055768010986)
(-146.93667225299998, 66.541629845, -145.93830839299997, 66.65413196100008)
(-141.66447917999994, 65.2905